In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from subprocess import call
from scipy.stats.distributions import gamma, lognorm
import json
import wget
import os
import os.path
from datetime import datetime
import pytz
from collections import OrderedDict
#from google.colab import drive
import math

from datetime import datetime, timedelta
from urllib.request import urlopen
import collections
import copy
#os.chdir('/content/gdrive/My Drive')
#drive.mount('/content/gdrive')

In [2]:
states = json.load(open('allmetrics_states.json'))

delhi = {}

delhi['Delhi'] = states['Delhi']
delhi['Chandigarh'] = states['Chandigarh']

delhi['datetime'] = states['datetime']

with open('delhi-demo.json','w') as file:
    json.dump(delhi,file)

In [3]:
states = json.load(open('delhi-demo.json'))

districts = list(states.keys())[:-1]

In [4]:
dataset=pd.read_csv('population.csv')
population=pd.DataFrame()
population["District"]=dataset['State']
population["Population"]=dataset['Population']
population = population.set_index('District')
population

,Population
District,
Delhi,18710922
Haryana,28204692
Kerala,35699443
Himachal Pradesh,7451955
Punjab,30141373
Telangana,39362732
Goa,1586250
Uttarakhand,11250858
Chandigarh,1158473


In [5]:
def convert_vacc(dat):
    try:
        return datetime.strptime(dat, '%d/%m/%Y').strftime('%d %B')
    except:
        return datetime.strptime(dat, '%d-%m-%Y').strftime('%d %B')


def get_daily_series(a):
    return [t - s for s, t in zip(a, a[1:])]


'''
def moving_avg(data, window_size=7):
    numbers_series = pd.Series(data)
    windows = numbers_series.rolling(window_size)
    moving_averages = windows.mean()
    moving_averages_list = moving_averages.tolist()
    return moving_averages_list[window_size - 1:]
'''

def moving_avg(data, window_size=7):
    ma = ['']*len(data)
    for w in range(window_size, len(data)):
        sum1 = 0
        count = 0
        for s in range(window_size):
            if (len(str(data[w - s])) != 0):
                sum1 += int(data[w - s])
                count += 1
        if count != 0:
            ma[w] = sum1 / count
    
    return ma


def rate_increase_cases(a, window_size=14):
    increase_rate = ['']*window_size
    
    for t,s in zip(a[window_size:],a):
        try:
            increase_rate.append((t - s) * 100 / t)
        except:
            increase_rate.append('')
            
    
    return increase_rate


def TRP_14ma(daily_pos, daily_tested, window_size=7):
    daily_pos_rate_ma = ['']*len(daily_pos)
    for w in range(window_size, len(daily_pos)):
        sum1 = 0
        sum2 = 0
        for s in range(window_size):
            if (len(str(daily_pos[w - s])) != 0
                    and len(str(daily_tested[w - s])) != 0):
                sum1 += int(daily_pos[w - s])
                sum2 += int(daily_tested[w - s])

        if sum2 != 0:
            daily_pos_rate_ma[w] = sum1 * 100 / abs(sum2)

    return daily_pos_rate_ma


def category_change(category):
    return [t == s for s, t in zip(category, category[1:])]

def divide_helper(a,b):
    replace = {'': 0}
    restore = {0:''}
    
    a = [replace.get(x, x) for x in a]
    a = np.array(a)/b
    
    return [restore.get(x, x) for x in a]

In [6]:
def logic(js):
    if js["rate_increase_new_cases"][i] >5 and js["TPR"][i] > 10 and js["pct_ICU"][i] < 40 and (js["daily_cases_per_million"][i]  >100 or js["daily_tests_per_million_14ma"][i] < 140):
        return "RED"
    elif (js["rate_increase_new_cases"][i] >= 2) and (js["TPR"][i] >=5) and (js["pct_ICU"][i] <= 80) and ((js["daily_cases_per_million"][i]  >= 20) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "GREEN"

    
def agg_logic(js):
    if js["rate_increase_new_cases"][i] < 2 and js["TPR"][i] < 5 and js["pct_ICU"][i] > 80 and (js["daily_cases_per_million"][i]  <20 or js["daily_tests_per_million_14ma"][i] > 140):
        return "GREEN"
    elif (js["rate_increase_new_cases"][i] <=5) and (js["TPR"][i] <= 10) and js["pct_ICU"][i] >= 40 and  ((js["daily_cases_per_million"][i]  <100) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "RED"

def logic_without_icu(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    elif js["rate_increase_new_cases"][i] >5 and js["TPR"][i] > 10 and (js["daily_cases_per_million"][i]  >100 or js["daily_tests_per_million_14ma"][i] < 140):
        return "RED"
    elif (js["rate_increase_new_cases"][i] >= 2) and (js["TPR"][i] >=5) and ((js["daily_cases_per_million"][i]  >= 20) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "GREEN"
    
    
def agg_logic_without_icu(js,i):
    if js["rate_increase_new_cases"][i] == '' or js["TPR"][i] == '' or js["daily_cases_per_million"][i] == '' or js["daily_tests_per_million_14ma"][i] == '':
        return "GRAY"
    elif js["rate_increase_new_cases"][i] < 2 and js["TPR"][i] < 5 and (js["daily_cases_per_million"][i]  <20 or js["daily_tests_per_million_14ma"][i] > 140):
        return "GREEN"
    elif (js["rate_increase_new_cases"][i] <=5) and (js["TPR"][i] <= 10) and ((js["daily_cases_per_million"][i]  <100) or (js["daily_tests_per_million_14ma"][i] < 140)):
        return "YELLOW"
    else:
        return "RED"

In [7]:
t = ['','','','','',3,2,6,2,7,2,7]

rate_increase_cases(t,3)

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 -50.0,
 71.42857142857143,
 -200.0,
 71.42857142857143]

In [8]:
for dist in districts:
    
    daily_pos = states[dist]['daily_positive_cases']
    daily_positive_cases_14ma = moving_avg(daily_pos,14)
    states[dist]['daily_positive_cases_14ma'] = daily_positive_cases_14ma
    
    daily_tested = states[dist]['daily_tests']
    daily_tests_14ma = moving_avg(daily_tested,14)
    states[dist]['daily_tests_14ma'] = daily_tests_14ma
    
    states[dist]['rate_increase_new_cases'] = rate_increase_cases(daily_positive_cases_14ma,14)
    
    states[dist]['TPR'] = TRP_14ma(daily_pos,daily_tested,14)
    
    states[dist]['daily_tests_per_million_14ma'] = divide_helper(daily_tests_14ma,int(population['Population']['Delhi']))
    states[dist]['pct_ICU'] = ['']  #TODO
    states[dist]['category'] = [logic_without_icu(states[dist], i) for i in range(len(states[dist]['dates']))] #TODO - outputted by some function
    states[dist]['year'] = ['']
    
    states[dist]['is_change'] = ['']+category_change(states[dist]['category']) #TODO - Bool value: 1 when category changed from previous date or not
    

In [9]:
with open('delhi-demo-outputs.json','w') as file:
    json.dump(states,file)